# NEST simulation of the cortico-cerebellar closed-loop experiment

Preparation of the environment and loading of all necessary libraries

In [ ]:
client = get_hbp_service_client()
client.storage.download_file("/79916/scaffold_network_BSP.hdf5", "scaffold_network_BSP.hdf5")
client.storage.download_file("/79916/scaffold_network_BSP.json", "scaffold_network_BSP.json")
client.storage.download_file("/79916/scaffold_network_BSP_static.json", "scaffold_network_BSP_static.json")

import os
os.environ["NEST_MODULE_PATH"] = "/home/jovyan/nest-simulator-2.18.0-build/lib/nest"
os.environ["SLI_PATH"] = "/home/jovyan/nest-simulator-2.18.0-build/share/nest/sli"
os.environ["LD_LIBRARY_PATH"] = "/home/jovyan/nest-simulator-2.18.0-build/lib/nest:/home/jovyan/bin/lib"
os.environ["PATH"] = "/home/jovyan/bin/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin"
os.environ["SPATIALINDEX_C_LIBRARY"] = "/home/jovyan/bin/lib/libspatialindex.so"
os.environ["PYTHONPATH"] = "/home/jovyan/extra-cereb-nest/Tests:/opt/amber18/lib/python3.6/site-packages/:/home/jovyan/.local/nrn/lib/python:"
import sys
sys.path.insert(0, "/home/jovyan/nest-simulator-2.18.0-build/lib/python3.6/site-packages/")
sys.path.append("/home/jovyan/extra-cereb-nest/Tests")

import nest
nest.Install("cerebmodule")
nest.Install("extracerebmodule")

import numpy as np
import world
from population_view import PopView
from world_populations import Planner, Cortex, SensoryIO, MotorIO, DirectDCN, InverseDCN
from scaffold.core import from_hdf5
from scaffold.output import HDF5Formatter
from scaffold.config import JSONConfig

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.close_figures=False # keep figures open in pyplot
import auxiliary_functions as aux
import trajectories
trajectories.save_file()

Definition of protocol variables

In [ ]:
trial_len = 300  # [ms]
CORES = 16  # We use all the 16 cores available
nest.set_verbosity("M_WARNING")
nest.ResetKernel()
nest.SetKernelStatus(
    {
        "local_num_threads": CORES,
        "total_num_virtual_procs": CORES,
        "resolution": 1.0,
        "overwrite_files": True,
    }
)

### CREATE BRAIN (Importing Cerebellum scaffold and creating other populations)

In [ ]:
# Reconfigure scaffold
static = True
filename_h5 = "scaffold_network_BSP.hdf5"
if static:
    filename_config = "scaffold_network_BSP_static.json"
else:
    filename_config = "scaffold_network_BSP.json"
reconfigured_obj = JSONConfig(filename_config)
HDF5Formatter.reconfigure(filename_h5, reconfigured_obj)

# Create scaffold_model from HDF5
scaffold_model = from_hdf5(filename_h5)
scaffold_model.configuration.verbosity = 1

# Get scaffold model populations
S_MF = scaffold_model.get_entities_by_type("mossy_fibers")
S_IO = scaffold_model.get_cells_by_type("io_cell")[:, 0]
S_DCN = scaffold_model.get_cells_by_type("dcn_cell")[:, 0]

uz_pos = scaffold_model.labels["microzone-positive"]
uz_neg = scaffold_model.labels["microzone-negative"]

S_DCNp = np.intersect1d(S_DCN, uz_pos)
S_DCNn = np.intersect1d(S_DCN, uz_neg)
S_IOp = np.intersect1d(S_IO, uz_pos)
S_IOn = np.intersect1d(S_IO, uz_neg)

# Prepare adapters
adapter_forward = scaffold_model.create_adapter("BSP_USECASE")
adapter_inverse = scaffold_model.create_adapter("BSP_USECASE")

adapter_forward.enable_multi("forward")
adapter_inverse.enable_multi("inverse")

adapter_forward.prepare()
adapter_inverse.prepare()

# Get NEST populations
f_IOp = adapter_forward.get_nest_ids(S_IOp)
f_IOn = adapter_forward.get_nest_ids(S_IOn)

i_IOp = adapter_inverse.get_nest_ids(S_IOp)
i_IOn = adapter_inverse.get_nest_ids(S_IOn)

f_DCNp = adapter_forward.get_nest_ids(S_DCNp)
f_DCNn = adapter_forward.get_nest_ids(S_DCNn)

i_DCNp = adapter_forward.get_nest_ids(S_DCNp)
i_DCNn = adapter_forward.get_nest_ids(S_DCNn)

f_MF = adapter_forward.get_nest_ids(S_MF)
i_MF = adapter_inverse.get_nest_ids(S_MF)

# Define population views
MF_number = len(f_MF)
IO_number = len(f_IOp) + len(f_IOn)

planner_pv = Planner(MF_number, prism=0.0, baseline_rate=10.0, gain_rate=2.0)
cortex_pv = Cortex(MF_number, rbf_sdev=MF_number/32, baseline_rate=200.0, gain_rate=5.0)

f_IO_pv = SensoryIO(f_IOp, f_IOn)  # External from the scaffold,
i_IO_pv = MotorIO(i_IOp, i_IOn)    # to be connected after

f_DCN_pv = DirectDCN(f_DCNp, f_DCNn)
i_DCN_pv = InverseDCN(i_DCNp, i_DCNn)

f_MF_pv = PopView(f_MF)
i_MF_pv = PopView(i_MF)


# Connections among different modules
conn_dict_one_to_one = {"rule": "one_to_one"}
nest.Connect(planner_pv.pop, cortex_pv.pop, conn_dict_one_to_one, {'weight': 1.0})

We simulate one "empty" trial to reach a steady-state situation for buffers in the cortex neurons

In [ ]:
trial_counter = 0
planner_pv.set_prism(0)
# Discard one trial
nest.Simulate(trial_len)
x = cortex_pv.integrate(trial_i=trial_counter)
trial_counter += 1

### Calibration of the control variables generated by the motor cortex module

Simulation of:
- 10 trials with prism perturbation = 0°
- 10 trials with prism perturbation = 10°
- Computation of the linear calibration
- Testing of the calibration with 10 trials with prism deviation = 25°

#### Calibration - Prism = 0°

In [ ]:
fig, ax = plt.subplots(3, 1, sharex="col")
xs = []
for i in range(10):
    nest.Simulate(trial_len)
    x = cortex_pv.integrate(trial_i=trial_counter)
    xs.append(x)
    ax[0].plot(cortex_pv.torques, "black")
    ax[1].plot(cortex_pv.vel, "black")
    ax[2].plot(cortex_pv.pos, "black")
    trial_counter += 1
    print("Simulated trial {} out of {}".format(i+1, 10),  end='\r')
x_0 = np.mean(xs)
xs_0 = xs

ax[0].set_ylabel("Torque")
ax[1].set_ylabel("Velocity")
ax[2].set_ylabel("Position")
ax[2].set_xlabel("Time [ms]")

#### Calibration - Prism = 10°

In [ ]:
planner_pv.set_prism(10)
# Discard one trial
nest.Simulate(trial_len)
x = cortex_pv.integrate(trial_i=trial_counter)
trial_counter += 1

xs = []
for i in range(10):
    nest.Simulate(trial_len)
    x = cortex_pv.integrate(trial_i=trial_counter)
    xs.append(x)
    ax[0].plot(cortex_pv.torques, "red")
    ax[1].plot(cortex_pv.vel, "red")
    ax[2].plot(cortex_pv.pos, "red")
    trial_counter += 1
    print("Simulated trial {} out of {}".format(i+1, 10),  end='\r')
x_10 = np.mean(xs)
xs_10 = xs

#### This is the calibration of the system

In [ ]:
print("Output with prism  0° x_0 = {} \nOutput with prism 10° x_10 = {}".format(x_0, x_10))
get_error = world.get_error_function(x_0, x_10)

#### Testing how the error is when the prism is set to 25

In [ ]:
planner_pv.set_prism(25)
# Discard one trial
nest.Simulate(trial_len)
x = cortex_pv.integrate(trial_i=trial_counter)
trial_counter += 1

xs = []
for i in range(10):
    nest.Simulate(trial_len)
    x = cortex_pv.integrate(trial_i=trial_counter)
    xs.append(x)
    print("Simulated trial {} out of {}. Calibrated Error: {}".format(i+1, 10, get_error(x)))
    ax[0].plot(cortex_pv.torques, "blue")
    ax[1].plot(cortex_pv.vel, "blue")
    ax[2].plot(cortex_pv.pos, "blue")
    trial_counter += 1
x_25 = np.mean(xs)
xs_25 = xs
print("Mean Calibrated Error: ", get_error(x_25))

#### Calibration results:
Torques, Velocities and Position plots in three conditions:
- Black, with prism = 0°
- Red, with prism = 10°
- Blue, with prism = 25°

It is possible to notice a range of variability given by the intrinsic noise of spike coding

In [ ]:
plt.show()

Check of the linearity of the calibrated positions

In [ ]:
plt.close("all")
plt.figure()
plt.scatter(np.zeros(10), get_error(xs_0), c="black")
plt.scatter(10 * np.ones(10), get_error(xs_10), c="red")
plt.scatter(25 * np.ones(10), get_error(xs_25), c="blue")
plt.plot([-5, 30], [-5, 30], '--', c="black")
plt.xlabel("Input prism deviation [deg]")
plt.ylabel("Output error computed [deg]")
plt.title("Calibrated errors")
plt.show()

Reset the Notebook kernel before continuing with a new simulation

In [ ]:
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
    
restartkernel()